# Stern-Gerlach Quantum Measurement Simulator
### A Solution Architecture Case Study

**Author:** Dr. Boris Kiefer · **Stack:** Python / Jupyter · **Repo:** github.com/boriskiefer/stern-gerlach-poc · **Linkedin:** linkedin.com/in/boris-kiefer-85089831/

> *This notebook is structured as a solution architecture engagement: customer context, problem framing, solution design, and a working POC with measurable outcomes — ready for hand-off to a technical team.*

---

## 1 · Customer

**Who:** Undergraduate students in a second-year modern physics or introductory quantum mechanics course.

**What they know:** Classical mechanics and electromagnetism. They understand that a charged particle in a magnetic field experiences a force, and that a distribution of randomly oriented magnetic moments should produce a *continuous spread* of deflections on a detector screen — proportional to $\cos\theta$ where $\theta$ is the angle between the moment and the field.

**What they need:** A concrete, visual experience of the moment classical physics fails — not as an abstract statement in a textbook, but as something they can interact with, break, and rebuild. The goal is not to teach the formalism first. The goal is to create the *question* that the formalism will later answer.

**Success looks like:** A student who can explain, why the Stern-Gerlach result cannot be classical.

## 2 · Problem

### The gap between intuition and experiment

In 1922, Stern and Gerlach sent silver atoms through an inhomogeneous magnetic field. The deflecting force is:

$$F_z = \mu_z \cdot \frac{\partial B}{\partial z}$$

Classical prediction: $\mu_z = \mu \cos\theta$, with $\theta$ uniformly distributed. Result: a continuous smear across the screen.

Observed result: **two discrete spots**.

This single image encodes three foundational principles that define quantum mechanics as a discipline:

| Principle | What the screen shows |
|---|---|
| **Discreteness** | Exactly two spots — $m_s = \pm\frac{1}{2}$. No continuum. |
| **Probabilistic outcome** | Each atom lands in one spot with $P = \frac{1}{2}$, not because of hidden variables, but because the pre-measurement state is a superposition. |
| **Measurement collapse** | An atom that hits the upper spot will, if re-measured along the same axis, always hit the upper spot. The measurement changes the state. |

### Why static diagrams are insufficient

Every textbook shows the two-spot result. The problem is that a static image does not let students *probe* the boundary between classical and quantum behaviour. They cannot ask: *what if the noise were larger? What if the field were weaker? At what point does the quantum signature disappear?* These are precisely the questions that build physical intuition — and they require an interactive model.

## 3 · Solution Design

### Architecture decisions

**Minimal stack.** The deliverable must run without configuration in any standard Jupyter environment (JupyterLab, Binder, Google Colab). Zero custom dependencies beyond the scientific Python baseline. This is a deliberate constraint: a POC that requires a 45-minute setup is not a POC.

**Classical toggle as the pedagogical core.** The most powerful teaching moment is not the quantum result — it is the *transition* between classical and quantum regimes. The simulator exposes this transition directly: switch to classical mode and the discrete spots dissolve into a cosine-weighted smear. The student controls the crossover. This is more durable than being told the classical prediction fails.

**Thermal noise as the decoherence dial.** Real atomic beams have transverse velocity spread from the oven source. When thermal noise $\sigma$ is large relative to the deflection, the two quantum spots overlap and the result is experimentally indistinguishable from the classical prediction. As $\sigma$ decreases — or field strength increases — the spots resolve. This parameter encodes a genuine physical story: quantum signatures are fragile, and their observability is a function of experimental conditions, not just of the underlying theory.

**Measurable metrics built in.** The simulator reports spot separation, overlap coefficient, and statistical significance at every parameter setting. These are not decorative — they are the handoff artefacts a technical team needs to validate the model against real experimental data.

### Parameter map

| Dial | Physics | Pedagogical role |
|---|---|---|
| **N particles** | Statistical sample size | Convergence — when is the pattern trustworthy? |
| **Field strength** | $\propto \partial B / \partial z$ | Spot separation — the primary experimental control |
| **Thermal noise $\sigma$** | Transverse velocity spread | The classical-quantum transition dial |
| **Screen distance** | Drift length post-magnet | Amplifies or compresses deflection |
| **Mode toggle** | Quantum vs. classical physics | Direct comparison — the gap made visible |

In [17]:
# ── Cell 1: Tools  ─────────────────────────────────────────────────────────────
# Encapsulated physics + plotting. No global state.
# Hand-off note: replace run_quantum / run_classical with real
# numerical integration (Runge-Kutta on the Bloch equations)
# for a research-grade version.
#
# Fixes applied:
#   1. run_quantum: thermal_noise → noise (parameter name); noise scaled by distance
#   2. run_classical: noise also scaled by distance for consistency
#   3. plot_results: fixed z_range so x-axis does not auto-scale with distance
#      — range is now symmetric around 0, set by field * distance * 1.6

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ipywidgets as widgets
from IPython.display import display
from scipy import stats


# ── Physics kernels ────────────────────────────────────────────────────────────

def run_quantum(n, field, noise, distance):
    """Spin-1/2: discrete eigenvalues ±1/2, equal probability.
    Both deflection and thermal spread scale with distance.
    """
    spins = np.random.choice([-0.5, 0.5], size=n)
    z = spins * field * distance + \
        np.random.normal(0, noise * distance, n)
    return z, spins


def run_classical(n, field, noise, distance):
    """Classical: mu_z = cos(theta), theta uniform on [0, pi].
    Produces a continuous cosine-weighted distribution.
    Noise also scaled by distance for physical consistency.
    """
    theta = np.random.uniform(0, np.pi, n)
    mu_z  = np.cos(theta)                          # ranges [-1, +1]
    z = mu_z * field * distance * 0.5 + \
        np.random.normal(0, noise * distance, n)
    spins = np.where(z >= 0, 0.5, -0.5)            # dummy — not physical
    return z, spins


# ── Metrics ────────────────────────────────────────────────────────────────────

def compute_metrics(z, spins, mode):
    """Returns a dict of reproducible, reportable metrics."""
    m = {}
    m['N'] = len(z)

    if mode == 'quantum':
        z_up   = z[spins ==  0.5]
        z_down = z[spins == -0.5]
        m['frac_up']   = len(z_up) / len(z)
        m['frac_down'] = len(z_down) / len(z)
        # chi-squared test: are counts consistent with 50/50?
        chi2, p = stats.chisquare([len(z_up), len(z_down)])
        m['chi2'] = chi2
        m['p_value'] = p
        # spot separation
        m['separation'] = abs(np.mean(z_up) - np.mean(z_down)) if len(z_up) and len(z_down) else 0
        # overlap: fraction of up-spot density in the down-spot region
        if len(z_up) > 1 and len(z_down) > 1:
            threshold = (np.mean(z_up) + np.mean(z_down)) / 2
            overlap = (np.sum(z_up < threshold) + np.sum(z_down > threshold)) / len(z)
        else:
            overlap = 1.0
        m['overlap'] = overlap
    else:
        m['frac_up']   = None
        m['frac_down'] = None
        m['chi2']      = None
        m['p_value']   = None
        m['separation'] = 0
        m['overlap']   = 1.0

    m['z_mean'] = np.mean(z)
    m['z_std']  = np.std(z)
    return m


# ── Plot ───────────────────────────────────────────────────────────────────────

def plot_results(z, spins, metrics, mode, field, distance, n_bins=80):
    """Fixed z_range: symmetric around 0, scaled by field*distance.
    This prevents the x-axis from auto-rescaling with distance,
    which would artificially make distributions appear sharper or broader.
    """
    BG, PANEL, BORDER = '#0d1117', '#161b22', '#30363d'
    BLUE, RED, GREEN  = '#58a6ff', '#f85149', '#3fb950'
    DIM, LIGHT        = '#8b949e', '#e6edf3'

    fig = plt.figure(figsize=(13, 5), facecolor=BG)
    gs  = gridspec.GridSpec(1, 3, figure=fig, width_ratios=[2.8, 0.9, 1.4], wspace=0.35)
    ax_hist = fig.add_subplot(gs[0])
    ax_dot  = fig.add_subplot(gs[1])
    ax_met  = fig.add_subplot(gs[2])

    for ax in [ax_hist, ax_dot, ax_met]:
        ax.set_facecolor(PANEL)
        for sp in ax.spines.values(): sp.set_edgecolor(BORDER)
        ax.tick_params(colors=DIM, labelsize=8)

    # Fixed range: scales with physics but does not chase the data distribution
    half_range = max(field * distance * 1.6, 0.5)
#    z_range = (-half_range, half_range)
    z_range = (-4, +4)
    bins = np.linspace(*z_range, n_bins)

    # ── histogram ──
    if mode == 'quantum':
        z_up   = z[spins ==  0.5]
        z_down = z[spins == -0.5]
        ax_hist.hist(z_up,   bins=bins, color=BLUE, alpha=0.85, label=r'$m_s=+\frac{1}{2}$')
        ax_hist.hist(z_down, bins=bins, color=RED,  alpha=0.85, label=r'$m_s=-\frac{1}{2}$')
        ax_hist.set_title('Quantum  —  two discrete spots', color=LIGHT, pad=8)
    else:
        ax_hist.hist(z, bins=bins, color=GREEN, alpha=0.80, label='classical smear')
        ax_hist.set_title('Classical  —  continuous distribution', color=LIGHT, pad=8)

    ax_hist.set_xlabel('Screen position  z  (a.u.)', color=DIM)
    ax_hist.set_ylabel('Impact count', color=DIM)
    ax_hist.set_xlim(z_range)
    ax_hist.legend(framealpha=0.2, labelcolor='white', fontsize=9)
    ax_hist.axvline(0, color=BORDER, lw=0.8, ls='--')

    # ── dot screen ──
    if mode == 'quantum':
        ax_dot.scatter(np.random.uniform(-0.4, 0.4, len(z_up)),   z_up,
                       s=0.9, color=BLUE, alpha=0.35)
        ax_dot.scatter(np.random.uniform(-0.4, 0.4, len(z_down)), z_down,
                       s=0.9, color=RED,  alpha=0.35)
    else:
        ax_dot.scatter(np.random.uniform(-0.4, 0.4, len(z)), z,
                       s=0.9, color=GREEN, alpha=0.25)

    ax_dot.set_xlim(-1, 1); ax_dot.set_ylim(z_range)
    ax_dot.set_xticks([])
    ax_dot.set_title('Screen', color=LIGHT, pad=8)
    ax_dot.set_ylabel('z (a.u.)', color=DIM)
    ax_dot.axhline(0, color=BORDER, lw=0.8, ls='--')

    # ── metrics panel ──
    ax_met.axis('off')
    ax_met.set_title('Metrics', color=LIGHT, pad=8)

    def row(y, label, value, color=DIM):
        ax_met.text(0.05, y, label, transform=ax_met.transAxes,
                    color=DIM, fontsize=12, va='top', fontfamily='monospace')
        ax_met.text(0.98, y, value, transform=ax_met.transAxes,
                    color=color, fontsize=12, va='top', ha='right',
                    fontfamily='monospace', fontweight='bold')

    row(0.92, 'N particles',  f"{metrics['N']:,}")
    row(0.80, 'z mean',       f"{metrics['z_mean']:+.3f}")
    row(0.70, 'z std',        f"{metrics['z_std']:.3f}")

    if mode == 'quantum':
        row(0.58, 'frac up',    f"{metrics['frac_up']:.3f}",  BLUE)
        row(0.48, 'frac down',  f"{metrics['frac_down']:.3f}", RED)
        row(0.36, 'separation', f"{metrics['separation']:.3f}", LIGHT)
        row(0.26, 'overlap',    f"{metrics['overlap']:.3f}",
            GREEN if metrics['overlap'] < 0.05 else '#f0883e' if metrics['overlap'] < 0.2 else RED)
        p = metrics['p_value']
        p_str  = f"{p:.3f}" if p >= 0.001 else "< 0.001"
        p_col  = GREEN if p > 0.05 else RED
        row(0.14, 'χ² p-value', p_str, p_col)
        ax_met.text(0.5, 0.04,
            '50/50 expected\np > 0.05 = consistent',
            transform=ax_met.transAxes, color=DIM, fontsize=7,
            ha='center', va='bottom', style='italic')
    else:
        row(0.58, 'distribution', 'cosine-wtd', GREEN)
        row(0.46, 'no eigenvalues', '—', DIM)
        row(0.34, 'no quantisation', '—', DIM)

    n = metrics['N']
    fig.suptitle(
        f"MODE: {'QUANTUM' if mode=='quantum' else 'CLASSICAL'}  "
        f"  N={n:,}",
        color=DIM, fontsize=9, y=1.01
    )
    plt.show()


print('✓  Cell 1 loaded — tools ready.')

✓  Cell 1 loaded — tools ready.


In [18]:
# ── Cell 2: Simulator  ─────────────────────────────────────────────────────────
# Interactive POC — adjust dials, observe outcomes, read metrics.
#
# Suggested classroom sequence:
#   1. Start: Classical mode, noise=0.05, field=1.0  → smooth smear
#   2. Switch to Quantum                              → two spots appear
#   3. Raise noise to 0.6                             → spots merge, quantum
#      signature washes out — discuss decoherence
#   4. Increase field to 3.0, lower noise to 0.05    → spots sharply resolved,
#      overlap < 0.01, chi² p > 0.05 (fair coin)
#   5. Vary distance: spots broaden AND separate
#      together — overlap stays roughly constant
#   6. Vary N: watch metrics stabilise above N~2000

style  = {'description_width': '180px'}
layout = widgets.Layout(width='460px')

w_mode = widgets.ToggleButtons(
    options=[('⚛  Quantum', 'quantum'), ('〰  Classical', 'classical')],
    value='quantum',
    description='Mode',
    style={'description_width': '50px',
           'button_width': '140px'})

w_n     = widgets.IntSlider(value=2000, min=100, max=20000, step=100,
                             description='N particles',
                             style=style, layout=layout, continuous_update=False)
w_field = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1,
                               description='Field strength  ∂B/∂z',
                               style=style, layout=layout, continuous_update=False)
w_noise = widgets.FloatSlider(value=0.10, min=0.0, max=1.0, step=0.01,
                               description='Thermal noise  σ',
                               style=style, layout=layout, continuous_update=False)
w_dist  = widgets.FloatSlider(value=1.0, min=0.2, max=4.0, step=0.1,
                               description='Screen distance',
                               style=style, layout=layout, continuous_update=False)

def simulate(mode, n, field, noise, dist):
    fn = run_quantum if mode == 'quantum' else run_classical
    z, spins = fn(n, field, noise, dist)
    metrics  = compute_metrics(z, spins, mode)
    plot_results(z, spins, metrics, mode, field, dist)   # field + dist passed

out = widgets.interactive_output(
    simulate,
    {'mode': w_mode, 'n': w_n, 'field': w_field, 'noise': w_noise, 'dist': w_dist}
)

display(
    widgets.VBox([
        w_mode,
        widgets.HTML('<hr style="border-color:#30363d;margin:6px 0">'),
        w_n, w_field, w_noise, w_dist
    ]),
    out
)

Output()

---
## 5 · Hand-off Notes for Technical Team

This POC is intentionally minimal. The following extensions are natural next steps for a production or research-grade version:

**Physics fidelity.** Replace `run_quantum` with numerical integration of the time-dependent Schrödinger equation through the field region (e.g., split-operator method on a 2D grid). This gives realistic trajectory curvature and models the finite transit time through the magnet.

**Higher spins.** Parameterise on $s \in \{\tfrac{1}{2}, 1, \tfrac{3}{2}, 2\}$ to show $2s+1$ spots. The architecture of `run_quantum` already supports this with a one-line change to `np.random.choice`.

**Sequential measurements.** Chain two Stern-Gerlach stages with a variable relative angle. This is the canonical demonstration that quantum measurement is non-commutative — and the natural bridge to the qubit and quantum computing.

**Reproducibility.** All runs can be seeded via `np.random.seed(n)` for exact reproducibility. Add a seed widget to make published results fully reproducible from the notebook alone.

**Validation.** The `chi2 / p_value` metric in the metrics panel is the primary quantitative check. A production version should compare simulated spot positions against the 1922 Stern-Gerlach data (Zeitschrift für Physik, 1922) and against modern tabletop reproductions.

---
*Built as a solution architecture showcase. Structure: customer → problem → solution design → working POC → measurable metrics → technical hand-off.*